# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [5]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the SQL schema
sql_schema = load_artifact("artifacts/convisoft_schema.sql")
if not sql_schema:
    print("Warning: Could not load convisoft_schema.sql.")
else:
    print("Successfully loaded convisoft_schema.sql")

 # Load PRD
prd = load_artifact("artifacts/convisoft_prd.md")
if not prd:
    print("Warning: Could not load convisoft_prd.md.")
else:
        print("Successfully loaded convisoft_prd.md")

orm_models = load_artifact("artifacts/convisoft_db.py")
if not orm_models:
    print("Warning: Could not load convisoft_db.py.")
else:
    print("Successfully loaded convisoft_db.py")

2025-10-02 14:02:05,088 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


Successfully loaded convisoft_schema.sql
Successfully loaded convisoft_prd.md
Successfully loaded convisoft_db.py


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [8]:
# TODO: Write a prompt to generate a complete FastAPI application with in-memory data storage.
in_memory_api_prompt = f"""
As as Senior Python developer and using the following SQL schema: {sql_schema} PRD: {prd} and ORM models {orm_models}, 
generate a complete FastAPI application with in-memory data storage.
Generate these items:
1. Pydantic Models: For API data validation (request/response bodies).
2. FastAPI Endpoints: CRUD operations for each endpoint defined in the PRD.
3. In-Memory Data Storage: A simple Python list to act as as temporary, fake database for testing.

"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/convisoft_API_in_memory.py", overwrite=True)
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
# main.py
import uvicorn
from fastapi import FastAPI, HTTPException, Query, status, Body
from pydantic import BaseModel, Field, HttpUrl
from typing import List, Optional, Annotated
from datetime import date, datetime
from decimal import Decimal
import random

# --------------------------------------------------------------------------- #
# 1. Application Setup
# --------------------------------------------------------------------------- #
app = FastAPI(
    title="Contracting Visualization API",
    description="An API for exploring and analyzing US government contract data.",
    version="1.0.0",
)

# --------------------------------------------------------------------------- #
# 2. In-Memory Data Storage (Fake Database)
# --------------------------------------------------------------------------- #
# Using dictionaries for easy ID-based lookups
db = {
    "locations": {},
    "companies": {},
    "contracts": {},
    "naics_codes

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [9]:
# Prompt to generate SQLAlchemy models and the database session/dependency code.
db_code_prompt = f"""
Using the following SQL schema: {sql_schema}
Generate these two items:
1. SQLAlchemy Models: to map to the tables in the schema.
2. Database Session/Dependency: boilerplate code to create a db engine, sessionmaker, and a FastAPI dependency function(get_db) to provide db sessions to endpoints.

Output should be in two distinct code blocks, one for the models and one for the session/dependency code.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
Here are the requested SQLAlchemy models and the database session/dependency boilerplate code.

### 1. SQLAlchemy Models

This code block defines Python classes that map to your SQL schema using SQLAlchemy's ORM.

```python
# models.py

from sqlalchemy import (
    Column,
    Integer,
    String,
    Float,
    Numeric,
    ForeignKey,
    Table,
    func
)
from sqlalchemy.orm import relationship, declarative_base

# Define the base class for declarative models
Base = declarative_base()

# --------------------------------------------------------------------
# 5. Junction Tables for M:N Relationships (as SQLAlchemy Table objects)
# --------------------------------------------------------------------

# Association table for Contracts and NAICS Codes
contract_naics_association = Table(
    'contract_naics',
    Base.metadata,
    Column('contract_id', Integer, ForeignKey('contracts.contract_id', onupd

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.